Crear canva

In [2]:

!pip install torch
!pip install ipycanvas
!pip install jupyterlab_widgets
!pip install torchvision

In [28]:
import os
import glob
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

In [6]:
# Esta celda realiza las siguientes operaciones:
# - Configura un lienzo interactivo para dibujar formas.
# - Implementa funciones para capturar dibujos y procesarlos.

from ipycanvas import Canvas # Proporciona la funcionalidad para crear lienzos interactivos en Jupyter Notebook.
from PIL import Image, ImageDraw # Se utiliza para trabajar con imágenes, como convertir de matrices a imágenes y viceversa.
import numpy as np # Es esencial para realizar operaciones matemáticas en arreglos (como las imágenes que se representan como matrices).
import os # Permite interactuar con el sistema operativo, como crear directorios.

# Se crea un lienzo blanco de 200x200 píxeles.
canvas = Canvas(width=200, height=200, background_color="white", sync_image_data = True)
# La opción sync_image_data = True asegura que los cambios en el lienzo se reflejen en la representación de la imagen.

# Función para capturar el dibujo como imagen
def get_drawing(): # Obtiene los datos de la imagen del lienzo.
    # Convierte la imagen del lienzo a una matriz NumPy y luego a imagen de escala de grises    
    img = Image.fromarray(canvas.get_image_data(0, 0, 200, 200))
    # Convertir la imagen a escala de grises (L)
    img = img.convert("L")
    img = img.resize((28, 28))  # Redimensionar a 28x28 píxeles
    # Convertir la imagen en escala de grises a una matriz NumPy
    return np.array(img)

# Función para guardar el dibujo
def save_drawing(class_name, count): # Crea un directorio para almacenar las imágenes de la clase especificada (por ejemplo, "0", "1", "2" para dígitos).
    # Crear directorio si no existe
    os.makedirs(f"data/{class_name}", exist_ok=True) # exist_ok=True evita errores si el directorio ya existe.
    # Obtiene el dibujo del lienzo utilizando get_drawing().
    img = get_drawing()
    
    # Guardar la imagen en el directorio especificado
    filepath = f"data/{class_name}/{count}.png"
    Image.fromarray(img).save(filepath)
    print(f"Dibujo guardado en: {filepath}")


# Variable para almacenar la última posición
last_x, last_y = None, None

# Función para dibujar en el lienzo
def on_mouse_down(x, y): 
    global last_x, last_y
    canvas.fill_style = "black"
    last_x, last_y = x, y  # Guardar la posición inicial cuando se presiona el botón del mouse

def on_mouse_move(x, y): # Se definen funciones para manejar los eventos de clic, movimiento y liberación del mouse.
    global last_x, last_y
    if last_x is not None and last_y is not None:
        canvas.stroke_style = "black"
        canvas.line_width = 5
        canvas.begin_path()
        canvas.move_to(last_x, last_y)
        canvas.line_to(x, y)
        canvas.stroke()
        last_x, last_y = x, y  # Actualizar la posición de la última coordenada

def on_mouse_up(x, y):
    global last_x, last_y
    last_x, last_y = None, None  # Resetear cuando se suelta el mouse

# Asignar los eventos de mouse al lienzo
canvas.on_mouse_down(on_mouse_down)
canvas.on_mouse_move(on_mouse_move)
canvas.on_mouse_up(on_mouse_up)

# Mostrar el lienzo
display(canvas)


Canvas(height=200, sync_image_data=True, width=200)

In [50]:
# Se guarda imagen en un directorio específico.
save_drawing("square", 7)

Dibujo guardado en: data/square/7.png


Generar Imágenes

In [52]:
# Esta celda realiza las siguientes operaciones:
# - Genera datos sintéticos de varias formas geométricas (círculos, cuadrados, etc.).
# - Guarda estas imágenes en una estructura de carpetas.

# Generar formas sintéticas

def generate_synthetic_data(class_name, count): # Esta función genera y guarda imágenes de formas geométricas básicas en blanco y negro.
    os.makedirs(f"data/{class_name}", exist_ok=True)
    for i in range(count):
        # Crear una imagen en blanco
        img = Image.new("L", (28, 28), "white") # Imagen de  28X28 píxeles en escala de grises "L"
        draw = ImageDraw.Draw(img) # Crea un objeto ImageDraw.Draw para dibujar en la imagen.

        # Nombre de la clase de la forma geométrica a generar. Debe ser "circle", "square", "triangle" o "star"
        if class_name == "circle":
            draw.ellipse((5, 5, 23, 23), outline="black", fill="black")
        elif class_name == "square":
            draw.rectangle((5, 5, 23, 23), outline="black", fill="black")
        elif class_name == "triangle":
            draw.polygon([(14, 5), (5, 23), (23, 23)], outline="black", fill="black")
        elif class_name == "star":
            draw.polygon([(14, 5), (10, 20), (5, 14), (23, 14), (18, 20)], outline="black", fill="black")

        # Guardar la imagen
        filepath = f"data/{class_name}/{i}.png" # Ruta del archivo 
        img.save(filepath) # Acá se guarda la imagen en la ruta 
        #print(f"Dibujo sintético guardado en: {filepath}")

# Generar 100 imágenes sintéticas por clase
generate_synthetic_data("circle", 100)
generate_synthetic_data("square", 100)
generate_synthetic_data("triangle", 100)
generate_synthetic_data("star", 100)


Cargar imágenes en el Dataset

In [54]:
# Esta celda realiza las siguientes operaciones:
# - Define una clase personalizada para cargar imágenes en un dataset.
# - Aplica transformaciones como normalización y conversión a tensor.

import os
import glob
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image

class DrawingDataset(Dataset): # root_dir es la ruta del directorio raíz que contiene las imágenes del dataset.
    def __init__(self, root_dir, transform=None): # Metodo que inicializa el dataset, verifica exisitencia de la imagen 
        self.root_dir = root_dir
        self.transform = transform # Del objeto de transforms de torchvision para transformaciones que se aplicarán a las imágenes cargadas.
        self.filepaths = glob.glob(os.path.join(root_dir, "*", "*.png")) # Lista de rutas de las imágenes en el dataset

        if not self.filepaths:
            raise ValueError(f"No se encontraron imágenes en {root_dir}. Verifica la estructura del directorio.")
            
        # Lista de etiquetas correspondientes a cada imagen, basadas en el nombre del directorio padre
        self.labels = [os.path.basename(os.path.dirname(path)) for path in self.filepaths]
        # Diccionario que mapea cada etiqueta única a un índice entero.
        self.label_to_idx = {label: idx for idx, label in enumerate(set(self.labels))}

    # Devuelve el tamaño del dataset (número total de imágenes).
    def __len__(self):
        return len(self.filepaths)

    # Obtiene una imagen y su etiqueta correspondiente a un índice específico
    def __getitem__(self, idx):
            # Abrir imagen y convertirla a escala de grises
            img = Image.open(self.filepaths[idx]).convert("L")  # Convertir a escala de grises (1 canal)
            label = self.label_to_idx[self.labels[idx]] # Obtiene la etiqueta del diccionario
            
            # Aplicar transformaciones
            if self.transform:
                img = self.transform(img)
            else:
                # Si no hay transformaciones, convierte la imagen a un tensor usando
                img = transforms.ToTensor()(img)
            
            return img, label # Devuelve una tupla con la imagen (como tensor) y su etiqueta (como entero).
# Transformaciones para normalizar imágenes
transform = transforms.Compose([
    transforms.Resize((28, 28)),       # Asegurar tamaño 28x28
    transforms.ToTensor(),            # Convertir a tensor
    transforms.Normalize((0.5,), (0.5,))  # Normalizar
])

# Cargar el dataset
dataset = DrawingDataset(root_dir="data", transform=transform)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)


Paso 3: Crear y Entrenar el Modelo CNN Definimos un modelo básico en PyTorch.

In [56]:
# Definimos la clase ImprovedCNN, que hereda de nn.Module para construir una red neuronal personalizada
class ImprovedCNN(nn.Module):  
    # Método constructor para inicializar los componentes de la red
    def __init__(self, num_classes):  
        # Llama al constructor de la clase base nn.Module
        super(ImprovedCNN, self).__init__()  
        # Primera capa convolucional con 1 canal de entrada, 32 filtros, tamaño de kernel 3x3, y padding de 1
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)  
        # Normalización por lotes para los 32 mapas de características de la primera capa convolucional
        self.bn1 = nn.BatchNorm2d(32)  
        # Segunda capa convolucional con 32 canales de entrada, 64 filtros, y configuración similar
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)  
        # Normalización por lotes para los 64 mapas de características
        self.bn2 = nn.BatchNorm2d(64)  
        # Tercera capa convolucional con 64 canales de entrada y 128 filtros
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)  
        # Normalización por lotes para los 128 mapas de características
        self.bn3 = nn.BatchNorm2d(128)  
        # Capa de agrupamiento máximo con tamaño de ventana 2x2 y salto de 2
        self.pool = nn.MaxPool2d(2, 2)  
        # Primera capa completamente conectada, reduce a 256 neuronas (tamaño de entrada depende de la salida convolucional)
        self.fc1 = nn.Linear(128 * 3 * 3, 256)  
        # Segunda capa completamente conectada para clasificar en el número de clases especificado
        self.fc2 = nn.Linear(256, num_classes)  
        # Capa Dropout para prevenir sobreajuste, desactivando el 50% de las neuronas durante el entrenamiento
        self.dropout = nn.Dropout(0.5)  

    # Método forward para definir cómo los datos pasan por las capas de la red
    def forward(self, x):  
        # Pasa los datos por la primera capa convolucional, BatchNorm, ReLU, y luego por MaxPooling
        x = self.pool(F.relu(self.bn1(self.conv1(x))))  
        # Pasa los datos por la segunda capa convolucional, BatchNorm, ReLU, y luego por MaxPooling
        x = self.pool(F.relu(self.bn2(self.conv2(x))))  
        # Pasa los datos por la tercera capa convolucional, BatchNorm, ReLU, y luego por MaxPooling
        x = self.pool(F.relu(self.bn3(self.conv3(x))))  
        # Reorganiza los datos en una forma plana para la entrada a las capas completamente conectadas
        x = x.view(x.size(0), -1)  
        # Pasa los datos por la primera capa completamente conectada y aplica la función de activación ReLU
        x = F.relu(self.fc1(x))  
        # Aplica Dropout para prevenir sobreajuste
        x = self.dropout(x)  
        # Pasa los datos por la segunda capa completamente conectada para producir la salida final
        x = self.fc2(x)  
        # Devuelve la salida de la red
        return x  

# Obtiene el número de clases del conjunto de datos usando su mapeo de etiquetas a índices
num_classes = len(dataset.label_to_idx)  
# Crea una instancia del modelo ImprovedCNN, enviándola al dispositivo adecuado (GPU si está disponible, de lo contrario CPU)
model = ImprovedCNN(num_classes).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))  


Entrenamiento del modelo

In [58]:
# Define una función para entrenar el modelo, tomando como parámetros el modelo, dataloader, función de pérdida, optimizador, y el número de épocas
def train_model(model, dataloader, criterion, optimizer, epochs=5):  
    # Detecta si hay una GPU disponible y la usa; de lo contrario, utiliza la CPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  
    # Envía el modelo al dispositivo (GPU o CPU)
    model.to(device)  
    # Lista para almacenar los valores de pérdida de cada época
    loss_values = []  
    # Lista para almacenar la precisión de cada época
    accuracy_values = []  

    # Itera a través del número especificado de épocas
    for epoch in range(epochs):  
        # Cambia el modelo al modo de entrenamiento (habilita dropout y batch norm para entrenamiento)
        model.train()  
        # Inicializa acumuladores para la pérdida y métricas de precisión
        running_loss = 0.0  
        correct = 0  # Número de predicciones correctas
        total = 0  # Número total de muestras procesadas

        # Itera a través del dataloader para procesar lotes de imágenes y etiquetas
        for images, labels in dataloader:  
            # Envía las imágenes y etiquetas al dispositivo (GPU o CPU)
            images, labels = images.to(device), labels.to(device)  

            # Reinicia los gradientes acumulados del optimizador
            optimizer.zero_grad()  
            # Pasa las imágenes a través del modelo para obtener las predicciones
            outputs = model(images)  
            # Calcula la pérdida entre las predicciones y las etiquetas verdaderas
            loss = criterion(outputs, labels)  
            # Calcula los gradientes para todos los parámetros del modelo
            loss.backward()  
            # Actualiza los parámetros del modelo usando los gradientes calculados
            optimizer.step()  

            # Suma la pérdida del lote actual al acumulador de pérdidas
            running_loss += loss.item()  
            # Obtiene las predicciones de clase más probables para cada muestra
            _, predicted = torch.max(outputs, 1)  
            # Incrementa el conteo total de muestras procesadas
            total += labels.size(0)  
            # Incrementa el conteo de predicciones correctas comparándolas con las etiquetas reales
            correct += (predicted == labels).sum().item()  

        # Calcula la pérdida promedio de la época
        epoch_loss = running_loss / len(dataloader)  
        # Calcula la precisión promedio de la época
        epoch_accuracy = 100 * correct / total  
        # Almacena la pérdida promedio de la época en la lista de pérdidas
        loss_values.append(epoch_loss)  
        # Almacena la precisión promedio de la época en la lista de precisiones
        accuracy_values.append(epoch_accuracy)  

        # Imprime los resultados de la época actual: pérdida promedio y precisión promedio
        print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")  

    # Devuelve las listas de pérdida y precisión para todas las épocas
    return loss_values, accuracy_values  

# Inicializa la función de pérdida como entropía cruzada para clasificación multiclase
criterion = nn.CrossEntropyLoss()  
# Inicializa el optimizador Adam con los parámetros del modelo y una tasa de aprendizaje de 0.001
optimizer = optim.Adam(model.parameters(), lr=0.001)  

# Llama a la función train_model para entrenar el modelo con el dataloader, criterio, optimizador y 5 épocas
loss_values, accuracy_values = train_model(model, dataloader, criterion, optimizer, epochs=5)  


Epoch 1/5, Loss: 0.1569, Accuracy: 93.75%
Epoch 2/5, Loss: 0.0002, Accuracy: 100.00%
Epoch 3/5, Loss: 0.0002, Accuracy: 100.00%
Epoch 4/5, Loss: 0.0001, Accuracy: 100.00%
Epoch 5/5, Loss: 0.0001, Accuracy: 100.00%


In [46]:
display(canvas)

Canvas(height=200, image_data=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\xc8\x00\x00\x00\xc8\x08\x06\x0…

Entrenamos el modelo con los datos capturados.

In [60]:
# Esta celda realiza las siguientes operaciones:
# - Utiliza un modelo previamente entrenado para predecir la clase de un dibujo en el lienzo.
# - Limpia el lienzo después de realizar una predicción.

import torch
from torchvision import transforms
from PIL import Image
import numpy as np

# Crear la transformación que se debe aplicar al dibujo
transform = transforms.Compose([
    transforms.ToTensor(),                  # Convierte a tensor
    transforms.Normalize((0.5,), (0.5,))    # Normaliza (ajusta según las necesidades del modelo)
])

# Función para obtener la imagen desde el lienzo
def get_drawing():
    img = Image.fromarray(canvas.get_image_data(0, 0, 200, 200))
    img = img.convert("L")  # Convertir a escala de grises
    img = img.resize((28, 28))  # Redimensionar a 28x28 píxeles
    return np.array(img)  # Retorna como un array NumPy

# Función para predecir usando el modelo
def predict_drawing(model, dataset):
    # Obtener el dibujo actual del lienzo
    img = get_drawing()
    
    # Preprocesar la imagen
    img_tensor = transform(Image.fromarray(img)).unsqueeze(0)  # Agregar dimensión batch
    
    # Enviar la imagen al modelo para predicción
    model.eval()  # Establecer el modelo en modo evaluación
    with torch.no_grad():  # Desactivar gradientes para predicción
        output = model(img_tensor)  # Obtener las predicciones
        pred = torch.argmax(output, dim=1).item()  # Obtener la clase predicha
    
    # Mapear la predicción a la etiqueta correspondiente
    label = list(dataset.label_to_idx.keys())[list(dataset.label_to_idx.values()).index(pred)]
    print(f"Predicción: {label}")

# Ejemplo de uso después de dibujar algo en el lienzo:
predict_drawing(model, dataset)

#Limpiar el canvas después de la prediccion
canvas.clear()



Predicción: square
